In [52]:
# @title Import libraries
import pandas as pd
import re
import requests

In [42]:
# Scores data contains question metadata
scores = pd.read_csv('scores/bots_score_data_q4.csv')

fabQuestions = pd.read_csv('scores/Q4 2024 FAB - questions list - FINAL BOT.csv')

In [43]:
# Rename first column to 'link'
fabQuestions = fabQuestions.rename(columns={fabQuestions.columns[0]: 'link', 'title': 'question_title'})
scores = scores.rename(columns={'question_id': 'bot_question_id'})

In [44]:
def add_question_ids(df):
    def extract_metaculus_id(url):
        if pd.isna(url):
            return None
        pattern = r'/questions/(\d+)/'
        match = re.search(pattern, str(url))
        return match.group(1) if match else None
    
    # Create new column with extracted IDs
    df['cp_question_id'] = df['link'].apply(extract_metaculus_id)
    return df

fabQuestions = add_question_ids(fabQuestions)

# Take only the questions that were also on main feed (cp_question_id not None)
fabQuestions = fabQuestions[~fabQuestions['cp_question_id'].isna()]

In [45]:
# Merge scores and fabQuestions on question title
merged = scores[['bot_question_id', 'question_title', 'cp_reveal_time']].merge(fabQuestions[['question_title', 'cp_question_id']], on='question_title', how='right')

# Drop duplicates
merged = merged.drop_duplicates()

# Cast merged['bot_question_id'] to int
merged['bot_question_id'] = merged['bot_question_id'].astype('Int64')

In [46]:
# Show head of merged data where cp_question_id is not null
merged[~merged['cp_question_id'].isnull()].head()

,bot_question_id,question_title,cp_reveal_time,cp_question_id
0,28922,Will a senolytic therapy be approved for comme...,2024-10-22 14:30:00+00,1621
140,28923,Will a nuclear weapon be detonated as an act o...,2024-10-22 14:30:00+00,4779
285,28924,Will a book written by a language model make t...,2024-10-22 14:30:00+00,5587
425,28935,"On October 31, 2024, will Nvidia's market capi...",2024-10-22 14:30:00+00,24806
570,28959,Will the Liberal Democratic Party win a majori...,2024-10-23 14:30:00+00,28972


In [47]:
# Did every cp_question_id get a bot_question_id? Show me rows where bot_question_id is null
nomatch = merged[merged['bot_question_id'].isnull()]

nomatch.shape

(52, 4)

In [48]:
merged.shape

(154, 4)

In [ ]:
nomatch.head() # I think these don't match because they haven't resolved yet (i.e. no scores)

,bot_question_id,question_title,cp_reveal_time,cp_question_id
1705,<NA>,Will at least 200 Benin Bronzes go from the Br...,NaN,13924
1861,<NA>,Will the UK Abolish the Two-Child Benefit Cap ...,NaN,26616
2757,<NA>,Will the Russian government officially ban You...,NaN,28658
3553,<NA>,Will the U.S. have passed legislation that req...,NaN,17373
3554,<NA>,Will SpaceX re-use a Starship booster before J...,NaN,29222


In [51]:
# Save merged data to csv
merged.to_csv('bot_to_main_feed_ids.csv', index=False)